In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import tensorflow as tf
path = r'..\\embeddings\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 300
with open(path + "base_emb", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

148733


In [3]:
print(type(word_dict))

<class 'dict'>


In [ ]:
def find_candidate(sim,id_a,id_a_star,id_b):
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    
    if(myList[0] != 0):
        index1 = np.argmax(sim[:myList[0]])
    else:
        index1 = None
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim2)
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim3)
        index3 +=  myList[1]+1
    else:
        index3 = None
    if(myList[2]+1 < len(sim)):
        index4 = np.argmax(sim[myList[2]+1:])
        index4 +=  myList[2]+1
    else:
        index4 = None
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]

## 3cosAdd 

In [4]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)
def three_cos_predict_np(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix[id_a,:]
    weight_b = matrix[id_b,:]
    weight_a_star = matrix[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    
    return word_dict[index]

In [5]:
import time
startTime = time.time()

print(three_cos_predict_np('small','smaller','weak','weaker'))
print(three_cos_predict_np('big','bigger','small','smaller'))
print(three_cos_predict_np('germany','berlin','france','paris'))
print(three_cos_predict_np('one','two','three','four'))
print(three_cos_predict_np('tall','taller','big','bigger'))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

strong
larger
paris
four
showcase
Execution time in seconds: 0.13303017616271973


In [6]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list


print(find_nearest_k('smaller',7))#this should produce one, for the same vectors
print(find_nearest_k('computer',7))

[('smaller', 0.9999999999999999), ('larger', 0.7679766195587474), ('small', 0.6711319448786391), ('large', 0.6451574459607814), ('typically', 0.6236010111428973), ('these', 0.6218913987567888), ('contained', 0.620946589633534)]
[('computer', 1.0000000000000002), ('computers', 0.5085382386991789), ('design', 0.4626533925729936), ('systems', 0.45063344092741375), ('allows', 0.4436976593874406), ('advanced', 0.4278621239165861), ('utilizing', 0.4266367676316634)]


In [8]:
count_sucess = 0
count_unsucessful = 0
count_out_of_vocab = 0
count_b = 0
with open(r'..\datasets\miklov Analogies.txt') as file:
    lines = file.readlines()
    for index, line in enumerate(lines):
        if(index % 1000 == 0):
            print(str(index) + " of " + str(len(lines)))
            
        if line.startswith(":"):
            pass
        else:
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()#super important
            
            predicted = three_cos_predict_np(a,a_star,b,b_star)
            print(a,a_star,b,b_star,'=>',predicted)
            if(predicted == None):
                count_out_of_vocab += 1
            elif(predicted == b_star):
                count_sucess += 1
            elif(predicted == b):
                count_b += 1
            else:
                count_unsucessful += 1

print(count_sucess/ len(lines))
print(count_out_of_vocab)

0 of 13106
abuja nigeria accra ghana => malaysia
abuja nigeria algiers algeria => malaysia
abuja nigeria amman jordan => malaysia
abuja nigeria ankara turkey => lebanon
abuja nigeria antananarivo madagascar => subcontinent
abuja nigeria apia samoa => cyrus
abuja nigeria ashgabat turkmenistan => vallabh
abuja nigeria asmara eritrea => ethiopia
abuja nigeria astana kazakhstan => malaysia
abuja nigeria athens greece => occupies
abuja nigeria baghdad iraq => lebanon
abuja nigeria baku azerbaijan => lebanon
abuja nigeria bamako mali => implementation
abuja nigeria bangkok thailand => thereafter
abuja nigeria banjul gambia => implementation
abuja nigeria beijing china => tianjin
abuja nigeria beirut lebanon => lebanon
abuja nigeria belgrade serbia => lebanon
abuja nigeria belmopan belize => ethiopia
abuja nigeria berlin germany => hamburg
abuja nigeria bern switzerland => lebanon
abuja nigeria bishkek kyrgyzstan => malaysia
abuja nigeria bratislava slovakia => nowadays
abuja nigeria brussels

antananarivo madagascar ashgabat turkmenistan => liberia
antananarivo madagascar asmara eritrea => seychelles
antananarivo madagascar astana kazakhstan => plates
antananarivo madagascar athens greece => 1986
antananarivo madagascar baghdad iraq => africa
antananarivo madagascar baku azerbaijan => colombia
antananarivo madagascar bamako mali => party
antananarivo madagascar bangkok thailand => wherein
antananarivo madagascar banjul gambia => africa
antananarivo madagascar beijing china => 1986
antananarivo madagascar beirut lebanon => partisans
antananarivo madagascar belgrade serbia => 1986
antananarivo madagascar belmopan belize => utah
antananarivo madagascar berlin germany => munich
antananarivo madagascar bern switzerland => ciphers
antananarivo madagascar bishkek kyrgyzstan => coast
antananarivo madagascar bratislava slovakia => 1986
antananarivo madagascar brussels belgium => lisbon
antananarivo madagascar bucharest romania => 1986
antananarivo madagascar budapest hungary => 1986

astana kazakhstan dhaka bangladesh => turkey
astana kazakhstan doha qatar => ban
astana kazakhstan dublin ireland => ireland
astana kazakhstan dushanbe tajikistan => venezuela
astana kazakhstan funafuti tuvalu => sk
astana kazakhstan gaborone botswana => southeastern
astana kazakhstan georgetown guyana => sk
astana kazakhstan hanoi vietnam => ukraine
astana kazakhstan harare zimbabwe => mexico
astana kazakhstan havana cuba => mexico
astana kazakhstan helsinki finland => sweden
astana kazakhstan islamabad pakistan => rotated
astana kazakhstan jakarta indonesia => affecting
athens greece baghdad iraq => england
athens greece baku azerbaijan => lebanon
athens greece bamako mali => herzegovina
athens greece bangkok thailand => herzegovina
athens greece banjul gambia => intercedes
athens greece beijing china => china
athens greece beirut lebanon => lebanon
athens greece belgrade serbia => serbia
athens greece belmopan belize => ukraine
athens greece berlin germany => germany
athens greece b

bangkok thailand kabul afghanistan => chile
bangkok thailand kampala uganda => compliance
bangkok thailand kathmandu nepal => compliance
bangkok thailand khartoum sudan => chico
bangkok thailand kiev ukraine => guardianship
banjul gambia beijing china => china
banjul gambia beirut lebanon => ix
banjul gambia belgrade serbia => zagreb
banjul gambia belmopan belize => dolerite
banjul gambia berlin germany => munich
banjul gambia bern switzerland => dummy
banjul gambia bishkek kyrgyzstan => sexes
banjul gambia bratislava slovakia => sindh
banjul gambia brussels belgium => hiding
banjul gambia bucharest romania => zagreb
banjul gambia budapest hungary => 1934
banjul gambia bujumbura burundi => eligius
banjul gambia cairo egypt => states
banjul gambia canberra australia => australian
banjul gambia caracas venezuela => governed
banjul gambia chisinau moldova => memon
banjul gambia conakry guinea => tempered
banjul gambia copenhagen denmark => 1934
banjul gambia dakar senegal => governed
banj

belmopan belize lilongwe malawi => chongqing
belmopan belize lima peru => acknowledged
berlin germany bern switzerland => flee
berlin germany bishkek kyrgyzstan => drc
berlin germany bratislava slovakia => funded
berlin germany brussels belgium => bavaria
berlin germany bucharest romania => money
berlin germany budapest hungary => flee
berlin germany bujumbura burundi => achtung
berlin germany cairo egypt => residents
berlin germany canberra australia => australia
berlin germany caracas venezuela => abroad
berlin germany chisinau moldova => crore
berlin germany conakry guinea => pugs
berlin germany copenhagen denmark => abroad
berlin germany dakar senegal => usgs
berlin germany damascus syria => silently
berlin germany dhaka bangladesh => macclesfield
berlin germany doha qatar => discretization
berlin germany dublin ireland => ireland
berlin germany dushanbe tajikistan => countries
berlin germany funafuti tuvalu => seeadler
berlin germany gaborone botswana => burkina
berlin germany geo

...

groom bride boy girl => girl
groom bride brother sister => son
groom bride brothers sisters => name
groom bride dad mom => reason
groom bride father mother => son
groom bride grandfather grandmother => grandson
groom bride grandpa grandma => diabolical
groom bride grandson granddaughter => son
he she his her => her
he she husband wife => her
he she king queen => son
he she man woman => girl
he she nephew niece => husband
he she policeman policewoman => marry
he she prince princess => princess
he she son daughter => father
he she sons daughters => daughters
he she stepbrother stepsister => gladys
he she stepfather stepmother => disambiguation
he she stepson stepdaughter => herself
he she uncle aunt => husband
he she boy girl => girl
he she brother sister => son
he she brothers sisters => older
he she dad mom => let
he she father mother => mother
he she grandfather grandmother => husband
he she grandpa grandma => faster
he she grandson granddaughter => grandfather
he she groom bride => m

sons daughters groom bride => marry
sons daughters he she => she
sons daughters his her => her
sons daughters husband wife => wife
sons daughters king queen => ii
sons daughters man woman => woman
sons daughters nephew niece => wife
sons daughters policeman policewoman => marries
sons daughters prince princess => princess
sons daughters son daughter => daughter
stepbrother stepsister stepfather stepmother => fleeing
stepbrother stepsister stepson stepdaughter => spiritualist
stepbrother stepsister uncle aunt => mother
stepbrother stepsister boy girl => old
stepbrother stepsister brother sister => son
stepbrother stepsister brothers sisters => older
stepbrother stepsister dad mom => converts
stepbrother stepsister father mother => mother
stepbrother stepsister grandfather grandmother => abandoned
stepbrother stepsister grandpa grandma => yee
stepbrother stepsister grandson granddaughter => grandfather
stepbrother stepsister groom bride => hires
stepbrother stepsister he she => but
stepb

calm calmly apparent apparently => sexuality
cheerful cheerfully complete completely => contained
cheerful cheerfully efficient efficiently => crossbar
cheerful cheerfully fortunate fortunately => loudspeakers
cheerful cheerfully free freely => allow
cheerful cheerfully furious furiously => promptly
cheerful cheerfully happy happily => ayako
cheerful cheerfully immediate immediately => obtaining
cheerful cheerfully infrequent infrequently => streaming
cheerful cheerfully lucky luckily => chicks
cheerful cheerfully most mostly => many
cheerful cheerfully obvious obviously => intended
cheerful cheerfully occasional occasionally => numerous
cheerful cheerfully possible possibly => information
cheerful cheerfully precise precisely => information
cheerful cheerfully professional professionally => former
cheerful cheerfully quick quickly => fourteen
cheerful cheerfully quiet quietly => 1794
cheerful cheerfully rapid rapidly => operations
cheerful cheerfully rare rarely => cd
cheerful cheerfu

furious furiously safe safely => provide
furious furiously serious seriously => initial
furious furiously slow slowly => difficult
furious furiously sudden suddenly => unexpected
furious furiously swift swiftly => tokoroa
furious furiously typical typically => important
furious furiously unfortunate unfortunately => hemsley
furious furiously usual usually => settling
furious furiously amazing amazingly => yamagata
furious furiously apparent apparently => insufficient
furious furiously calm calmly => accumulate
furious furiously cheerful cheerfully => polite
furious furiously complete completely => initial
furious furiously efficient efficiently => accumulate
furious furiously fortunate fortunately => monferrato
furious furiously free freely => available
happy happily immediate immediately => obtained
happy happily infrequent infrequently => uncertainties
happy happily lucky luckily => reluctant
happy happily most mostly => many
happy happily obvious obviously => drastically
happy happi

KeyboardInterrupt: 

In [7]:
8870 / 13106

0.6767892568289333

In [8]:
import winsound
winsound.Beep(440, 500)

normalised before exclude

In [ ]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    index1 = np.argmax(sim[:myList[0]])
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim[myList[0]+1:myList[1]])
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim[myList[1]+1:myList[2]])
        index3 +=  myList[1]+1
    else:
        index3 = None
    index4 = np.argmax(sim[myList[2]+1:])
    index4 +=  myList[2]+1
    
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    
    return word_dict[index]

In [ ]:
count_sucess = 0
count_unsucessful = 0
count_out_of_vocab = 0
count_b = 0
with open(r'..\datasets\miklov Analogies.txt') as file:
    lines = file.readlines()
    for index, line in enumerate(lines):
        if(index % 1000 == 0):
            print(str(index) + " of " + str(len(lines)))
            
        if line.startswith(":"):
            pass
        else:
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()#super important
            
            predicted = three_cos_predict_np_norm(a,a_star,b,b_star)
            if(predicted == None):
                count_out_of_vocab += 1
            elif(predicted == b_star):
                count_sucess += 1
            elif(predicted == b):
                count_b += 1
            else:
                count_unsucessful += 1

print(count_sucess/ len(lines))
print(count_out_of_vocab)

## 3cos Mult

In [ ]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star)
    
    nominator = nominator * sim_01(matrix_normalized,weight_b) 
    donominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / donominator
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    index1 = np.argmax(sim[:myList[0]])
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim[myList[0]+1:myList[1]])
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim[myList[1]+1:myList[2]])
        index3 +=  myList[1]+1
    else:
        index3 = None
    index4 = np.argmax(sim[myList[2]+1:])
    index4 +=  myList[2]+1
    
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    
    return word_dict[index]

In [ ]:
import time
startTime = time.time()

print(predict_three_cos_mult('small','smaller','weak','weaker'))
print(predict_three_cos_mult('big','bigger','small','smaller'))
print(predict_three_cos_mult('germany','berlin','france','paris'))
print(predict_three_cos_mult('one','two','three','four'))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
count_sucess = 0
count_unsucessful = 0
count_out_of_vocab = 0
with open(r'..\datasets\miklov Analogies.txt') as file:
    lines = file.readlines()
    for index, line in enumerate(lines):
        if(index % 100 == 0):
            print(str(index) + " of " + str(len(lines)) +" => "+str( count_sucess))
            
        if line.startswith(":"):
            pass
        else:
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()#super important
            
            predicted = predict_three_cos_mult(a,a_star,b,b_star)
            if(predicted == None):
                count_out_of_vocab += 1
            elif(predicted == b_star):
                count_sucess += 1
            else:
                count_unsucessful += 1
print(print(count_sucess/ len(lines)))
print(count_out_of_vocab)

In [ ]:
import winsound
winsound.Beep(440, 500)